# Άσκηση #2, Κωδικοποίηση Shannon-Fano:
Θεωρήστε τις εικόνες αποχρώσεων του γκρι `airplane.jpg` και `bridge.jpg`.  
 
**Ζητούμενα:**

Α. Υλοποιήστε και εφαρμόστε τη μέθοδο συμπίεσης-κωδικοποίησης Shannon-Fano για τις εικόνες “airplane.jpg” και “bridge.jpg”. Να μη γίνει χρήση έτοιμων συναρτήσεων-
υλοποιήσεων της μεθόδου. 

Β. Για κάθε εικόνα παραθέστε: 
1) Τις κωδικές λέξεις που προκύπτουν (αντιστοιχία τιμών φωτεινότητας με δυαδικές κωδικές λέξεις), 
2) Το μέσο μήκος κωδικής λέξης, 
3) Την  τιμή της εντροπίας, και 
4) Το λόγο συμπίεσης που επιτυγχάνεται. 

Γ. Για κάθε εικόνα, υπολογίστε τον αντίστοιχο λόγο συμπίεσης που επιτυγχάνεται με χρήση της μεθόδου συμπίεσης-κωδικοποίησης Huffman. Συγκρίνετε και σχολιάστε τα αποτελέσματα.

In [5]:
%pip install numpy scikit-image huffman --quiet


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import skimage as ski
import huffman
def sort_symbols(probabilities):
    return sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

def split_symbols(symbols):
    total = sum([prob for _, prob in symbols])
    acc = 0
    split_index = 0

    for i, (_, prob) in enumerate(symbols):
        acc += prob
        if acc >= total / 2:
            split_index = i + 1
            break

    return symbols[:split_index], symbols[split_index:]

def assign_codes(symbols, prefix, code_dict):
    if len(symbols) == 1:
        symbol, _ = symbols[0]
        code_dict[symbol] = prefix
        return

    left, right = split_symbols(symbols)
    assign_codes(left, prefix + '0', code_dict)
    assign_codes(right, prefix + '1', code_dict)

def shannon_fano(probabilities):
    sorted_symbols = sort_symbols(probabilities)
    code_dict = {}
    assign_codes(sorted_symbols, '', code_dict)
    return code_dict

def calculate_probabilities(image):
    unique, counts = np.unique(image, return_counts=True)
    total_pixels = image.size
    probabilities = {int(symbol): count / total_pixels for symbol, count in zip(unique, counts)}
    return probabilities

def encode_image(image, code_dict):
    encoded = ''
    for pixel in image.flatten():
        encoded += code_dict[int(pixel)]
    return encoded

# Συνάρτηση υπολογισμού μέσου μήκους κωδικών λέξεων
def calculate_average_codeword_length(code_dict):
    return np.mean([len(code) for code in code_dict.values()])

# Συνάρτηση υπολογισμού εντροπίας
def calculate_entropy(probabilities):
    entropy = -sum(p * np.log2(p) for p in probabilities.values())
    return entropy

# Συνάρτηση υπολογισμού λόγου συμπίεσης
def calculate_compression_ratio(image, encoded_image):
    original_size = image.size * 8  # Μέγεθος εικόνας σε bits
    compressed_size = len(encoded_image)  # Μέγεθος συμπιεσμένης εικόνας σε bits
    compression_ratio = original_size / compressed_size
    return compression_ratio

# Φόρτωση εικόνας σε grayscale

image_airplane = ski.io.imread("https://github.com/KaratziasK/Digital-Image-Processing/blob/main/project-2/instructions/images-project-2/airplane.jpg?raw=true")
image_bridge = ski.io.imread("https://github.com/KaratziasK/Digital-Image-Processing/blob/main/project-2/instructions/images-project-2/bridge.jpg?raw=true")

# Υπολογισμός πιθανοτήτων pixel values
probabilities_airplane = calculate_probabilities(image_airplane)

# Δημιουργία Shannon-Fano codes
codes_airplane = shannon_fano(probabilities_airplane)

# Κωδικοποίηση εικόνας
encoded_image_airplane = encode_image(image_airplane, codes_airplane)


# Υπολογισμός πιθανοτήτων pixel values
probabilities_bridge = calculate_probabilities(image_bridge)

# Δημιουργία Shannon-Fano codes
codes_bridge = shannon_fano(probabilities_bridge)

# Κωδικοποίηση εικόνας
encoded_image_bridge = encode_image(image_bridge, codes_bridge)


print("Κωδικές λέξεις Shannon-Fano για την εικόνα airplane.jpg:")
for symbol, code in sorted(codes_airplane.items()):
    print(f"{symbol}: {code}")

avg_length_airplane = calculate_average_codeword_length(codes_airplane)
# Υπολογισμός εντροπίας
entropy_airplane = calculate_entropy(probabilities_airplane)

# Υπολογισμός λόγου συμπίεσης
compression_ratio_airplane = calculate_compression_ratio(image_airplane, encoded_image_airplane)

print(f"\nΜέσο μήκος κωδικών λέξεων (airplane.jpg): {avg_length_airplane:.2f} bits")
print(f"Εντροπία (airplane.jpg): {entropy_airplane:.2f} bits")
print(f"Λόγος συμπίεσης (airplane.jpg): {compression_ratio_airplane:.2f}")

print("\n====================================\n")

print("Κωδικές λέξεις Shannon-Fano για την εικόνα bridge.jpg:")
for symbol, code in sorted(codes_bridge.items()):
    print(f"{symbol}: {code}")

avg_length_bridge = calculate_average_codeword_length(codes_bridge)
# Υπολογισμός εντροπίας
entropy_bridge = calculate_entropy(probabilities_bridge)

# Υπολογισμός λόγου συμπίεσης
compression_ratio_bridge = calculate_compression_ratio(image_bridge, encoded_image_bridge)

print(f"\nΜέσο μήκος κωδικών λέξεων (bridge.jpg): {avg_length_bridge:.2f} bits")
print(f"Εντροπία (bridge.jpg): {entropy_bridge:.2f} bits")
print(f"Λόγος συμπίεσης (bridge.jpg): {compression_ratio_bridge:.2f}")


import huffman  # Εξαρτάται από την βιβλιοθήκη huffman

def encode_image_huffman(image, code_dict):
    encoded = ''
    for pixel in image.flatten():
        encoded += code_dict[pixel]  # Χρησιμοποιούμε το pixel σαν κλειδί στον Huffman codebook
    return encoded

# Υπολογισμός των πιθανοτήτων για Huffman (ακριβώς όπως για Shannon-Fano)
def calculate_compression_ratio_huffman(image, probabilities):

    symbol_weights = [(symbol, prob) for symbol, prob in probabilities.items()]

    # Δημιουργία κωδικοποίησης Huffman
    huffman_codebook = huffman.codebook(symbol_weights)
    
    # Κωδικοποίηση της εικόνας
    encoded_image_huffman = encode_image_huffman(image, huffman_codebook)
    
    # Υπολογισμός του λόγου συμπίεσης
    original_size = image.size * 8  # Μέγεθος εικόνας σε bits
    compressed_size = len(encoded_image_huffman)  # Μέγεθος συμπιεσμένης εικόνας σε bits
    compression_ratio = original_size / compressed_size
    return compression_ratio

# Υπολογισμός πιθανοτήτων pixel values για την εικόνα airplane (όπως πριν)
probabilities_airplane = calculate_probabilities(image_airplane)
# Υπολογισμός λόγου συμπίεσης για την εικόνα με Huffman
compression_ratio_huffman_airplane = calculate_compression_ratio_huffman(image_airplane, probabilities_airplane)

# Υπολογισμός πιθανοτήτων για την εικόνα bridge
probabilities_bridge = calculate_probabilities(image_bridge)
# Υπολογισμός λόγου συμπίεσης για την εικόνα με Huffman
compression_ratio_huffman_bridge = calculate_compression_ratio_huffman(image_bridge, probabilities_bridge)

# Εμφάνιση αποτελεσμάτων
print("\n====================================\n")
print(f"Λόγος συμπίεσης με Huffman (airplane.jpg): {compression_ratio_huffman_airplane:.2f}")
print(f"Λόγος συμπίεσης  με Huffman (bridge.jpg): {compression_ratio_huffman_bridge:.2f}")


Κωδικές λέξεις Shannon-Fano για την εικόνα airplane.jpg:
14: 1111111111111101
15: 1111111111111110
17: 1111111111111100
18: 11111111111101
19: 1111111111101
20: 11111111111000
21: 111111111101
22: 111111111001
23: 111111111010
24: 11111111011
25: 11111111001
26: 11111111010
27: 11111100011
28: 11111010001
29: 11111011001
30: 11111010100
31: 1111100111
32: 1111100010
33: 1111100101
34: 1111100110
35: 11111010000
36: 1111101011
37: 1111101101
38: 1111101001
39: 1111110011
40: 1111101110
41: 11111100010
42: 11111100000
43: 11111101100
44: 11111101101
45: 11111101001
46: 1111110101
47: 11111110110
48: 1111110111
49: 11111110000
50: 11111110111
51: 11111110010
52: 111111110001
53: 11111110100
54: 11111101000
55: 111111110000
56: 11111110001
57: 11111110101
58: 11111110011
59: 11111100101
60: 11111100100
61: 11111010101
62: 1111101111
63: 11111011000
64: 1111100100
65: 11111000001
66: 1111100011
67: 11111000000
68: 111101111
69: 1111011101
70: 1111011100
71: 1111011001
72: 111101101
73: 1111